# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom rule-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [1]:
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Define an environment function based on the example in the documentation.

In [4]:
import gymnasium as gym

def run_episode(agent, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
import numpy as np

def random_agent(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent)

Obs: [-1.3975144e-03  1.4128754e+00 -1.4157099e-01  8.6898841e-02
  1.6261818e-03  3.2067977e-02  0.0000000e+00  0.0000000e+00] -> Action: 3
Obs: [-2.6993752e-03  1.4142532e+00 -1.2934321e-01  6.1239701e-02
  7.9688645e-04 -1.6586592e-02  0.0000000e+00  0.0000000e+00] -> Action: 3
Obs: [-0.0039259   1.415036   -0.11990148  0.03478436 -0.0019249  -0.05444105
  0.          0.        ] -> Action: 3
Obs: [-0.0050828   1.4152203  -0.11115988  0.00818528 -0.00639716 -0.08945324
  0.          0.        ] -> Action: 0
Obs: [-0.00623951  1.414805   -0.11114635 -0.0184843  -0.01086835 -0.08943211
  0.          0.        ] -> Action: 0
Obs: [-0.00739594  1.41379    -0.11113296 -0.0451539  -0.01533923 -0.08942558
  0.          0.        ] -> Action: 1
Obs: [-0.00861502  1.4121732  -0.11897655 -0.07188666 -0.01823466 -0.05791395
  0.          0.        ] -> Action: 0
Obs: [-0.00983391  1.4099566  -0.11896843 -0.09855565 -0.02113033 -0.05791868
  0.          0.        ] -> Action: 0
Obs: [-0.0110528

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [6]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [7]:
def rocket_agent(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent)

Obs: [ 0.0024045   1.399184    0.24353144 -0.5216022  -0.00277938 -0.05516351
  0.          0.        ] -> Action: 2
Obs: [ 0.00463133  1.3874869   0.2263112  -0.5198788  -0.00636531 -0.0717251
  0.          0.        ] -> Action: 2
Obs: [ 0.00689764  1.3760706   0.23008725 -0.507414   -0.00977726 -0.06824557
  0.          0.        ] -> Action: 2
Obs: [ 0.00919838  1.3649547   0.23338673 -0.49406892 -0.01305282 -0.06551719
  0.          0.        ] -> Action: 2
Obs: [ 0.01136847  1.3539665   0.22097382 -0.48840296 -0.01698458 -0.07864229
  0.          0.        ] -> Action: 2
Obs: [ 0.01360226  1.3430051   0.22707483 -0.48722205 -0.02064484 -0.07321171
  0.          0.        ] -> Action: 2
Obs: [ 0.01566324  1.3320477   0.21067    -0.48707002 -0.02516938 -0.09049924
  0.          0.        ] -> Action: 2
Obs: [ 0.01783934  1.3211025   0.22167873 -0.4865264  -0.02919351 -0.08048986
  0.          0.        ] -> Action: 2
Obs: [ 0.01993237  1.3108175   0.21387789 -0.4572086  -0.03373018

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [8]:
import numpy as np

def run_episode_test(agent):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent) for _ in range(n)]

rewards = run_episodes(rocket_agent)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [65]:
# Code goes here